In [32]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import platform
import seaborn as sns
import os

In [33]:
sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

print(platform.system()) # 플랫폼 확인

if platform.system() == 'Windows': # Window
    matplotlib.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin': # Mac
    matplotlib.rc('font', family='AppleGothic')

Darwin


In [34]:
os.getcwd()

'/Users/chance/Desktop/LH_home/ksc'

In [35]:
folder = os.listdir()

file = []
for f in folder:
    if "_2022" in f:
        data = pd.read_csv("./" + f, encoding="cp949", skiprows=15)
        data = data[list(data.columns[:11])]
        file.append(data)

gajwa = pd.concat(file)
# gajwa = gajwa.dropna(axis=0)

In [36]:
gajwa.도로조건.unique()

array(['-', '8m미만', '25m미만', '12m미만', '25m이상', nan], dtype=object)

In [37]:
geo = pd.read_csv("./geocoding.csv")
gajwa["경도"] = geo.X
gajwa["위도"] = geo.Y

gajwa = gajwa.fillna({'건축년도':gajwa['건축년도'].mode()[0]})
gajwa = gajwa.fillna({'도로명':gajwa['도로명'].mode()[0]})
gajwa = gajwa.fillna({'도로조건':gajwa['도로명'].mode()[0]})

# 도로조건을 정수로 변환
road=[]
for a in gajwa["도로조건"]:
    if a=="-":
        a = "12"
    road.append(a.split("m")[0])
gajwa["도로조건"] = road


# 년월 column 생성
year=[]
month=[]
for d in gajwa["계약년월"]:
    day = pd.to_datetime(d, format="%Y%m")
    year.append(day.year)
    month.append(day.month)

gajwa["year"] = year
gajwa["month"] = month

built_day = [pd.to_datetime(d, format="%Y").year for d in gajwa["건축년도"]]
gajwa["건축년도"] = built_day

# 위치 column만들기
gajwa["location"] = gajwa.시군구 + " " + gajwa.도로명

# 계약년월 기준으로 전월세전환율 입력하기
rate = pd.read_csv("./rate.csv", encoding="cp949")
rate = rate.T
rate = rate[3:]

month = []
for r in rate.index:
    new = "".join(r.split("."))
    month.append(int(new))

rate["계약년월"] = month
rate.columns = ["전월세전환율", "계약년월"]
rate.head()

gajwa = pd.merge(left=gajwa, right=rate, how="inner", on="계약년월")

keep = [int("".join(k.split(","))) for k in gajwa["보증금(만원)"]]
gajwa["보증금(만원)"] = keep


# 전월세전환율 기준으로 보증금+월세의 가치를 전세로 치환하여 value column으로 만들어준다.
value=[round(gajwa["월세(만원)"][i] * 12 / (gajwa["전월세전환율"][i]/100) + gajwa["보증금(만원)"][i]) for i in range(len(gajwa))]
gajwa["전세전환가격"] = value
#gajwa["도로별 전세값평균"] = gajwa.groupby("도로명").전세전환가격.transform("mean")


# 도로명 결측치 제거
indexNames = []
for i in range(len(gajwa["도로명"])):
    if "번길" not in gajwa["도로명"][i]:
        indexNames.append(i)

gajwa.drop(indexNames, inplace=True)

gajwa

,시군구,번지,도로조건,계약면적(㎡),전월세구분,계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명,경도,위도,year,month,location,전월세전환율,전세전환가격
0,경상남도 진주시 가좌동,1***,12,12.0,월세,202003,5,1500,25,2007,진주대로404번길,128.105889,35.157466,2020,3,경상남도 진주시 가좌동 진주대로404번길,4.8,7750
1,경상남도 진주시 가좌동,1***,8,15.0,월세,202003,11,500,40,2016,가좌길78번길,128.105857,35.157800,2020,3,경상남도 진주시 가좌동 가좌길78번길,4.8,10500
5,경상남도 진주시 가좌동,5**,12,19.8,월세,202003,14,200,36,2011,내동로348번길,128.106121,35.158214,2020,3,경상남도 진주시 가좌동 내동로348번길,4.8,9200
6,경상남도 진주시 가좌동,1***,8,20.0,월세,202003,26,300,30,2016,가좌길48번길,128.105857,35.157800,2020,3,경상남도 진주시 가좌동 가좌길48번길,4.8,7800
7,경상남도 진주시 가좌동,1***,12,20.1,월세,202003,8,300,30,2014,진주대로404번길,128.105966,35.156961,2020,3,경상남도 진주시 가좌동 진주대로404번길,4.8,7800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5251,경상남도 진주시 호탄동,6**,12,33.0,월세,202011,19,100,23,2002,호탄길21번길,128.106121,35.158214,2020,11,경상남도 진주시 호탄동 호탄길21번길,4.7,5972
5253,경상남도 진주시 호탄동,6**,8,42.9,전세,202011,25,7500,0,2018,호탄길9번길,128.100104,35.147844,2020,11,경상남도 진주시 호탄동 호탄길9번길,4.7,7500
5254,경상남도 진주시 호탄동,9**,12,50.0,월세,202011,10,5000,30,2011,가호로61번길,128.109160,35.157148,2020,11,경상남도 진주시 호탄동 가호로61번길,4.7,12660
5255,경상남도 진주시 호탄동,6**,8,50.0,전세,202011,18,9000,0,2018,호탄길9번길,128.104882,35.150661,2020,11,경상남도 진주시 호탄동 호탄길9번길,4.7,9000


In [38]:
corr = gajwa.corr()
corr

,계약면적(㎡),계약년월,계약일,보증금(만원),월세(만원),건축년도,경도,위도,year,month,전세전환가격
계약면적(㎡),1.000000,-0.149413,-0.014312,0.449450,0.056091,-0.075128,-0.031188,-0.036129,-0.150664,0.098479,0.439190
계약년월,-0.149413,1.000000,-0.019595,-0.030070,-0.005450,0.175812,0.019498,-0.040761,0.999910,-0.028032,0.277357
계약일,-0.014312,-0.019595,1.000000,0.002483,-0.016596,0.012419,-0.005443,-0.002576,-0.020223,0.047498,-0.020224
보증금(만원),0.449450,-0.030070,0.002483,1.000000,-0.688701,0.082603,-0.000382,-0.008920,-0.031438,0.103046,0.016563
월세(만원),0.056091,-0.005450,-0.016596,-0.688701,1.000000,0.005268,-0.006970,-0.007256,-0.004681,-0.057145,0.651482
건축년도,-0.075128,0.175812,0.012419,0.082603,0.005268,1.000000,0.006669,-0.003610,0.175849,-0.008837,0.122126
경도,-0.031188,0.019498,-0.005443,-0.000382,-0.006970,0.006669,1.000000,-0.539547,0.019618,-0.009581,-0.004569
위도,-0.036129,-0.040761,-0.002576,-0.008920,-0.007256,-0.003610,-0.539547,1.000000,-0.040842,0.007454,-0.025105
year,-0.150664,0.999910,-0.020223,-0.031438,-0.004681,0.175849,0.019618,-0.040842,1.000000,-0.041432,0.276658
month,0.098479,-0.028032,0.047498,0.103046,-0.057145,-0.008837,-0.009581,0.007454,-0.041432,1.000000,0.042488


In [39]:
# # 전세 월세 구분 데이터셋
gajwa = gajwa[gajwa.전월세구분 == "월세"]
gajwa = pd.DataFrame.drop(gajwa, columns=["전월세구분"], axis=1)

#원핫인코딩
# gajwa = pd.get_dummies(gajwa, columns = ["도로조건"])

gajwa

,시군구,번지,도로조건,계약면적(㎡),계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명,경도,위도,year,month,location,전월세전환율,전세전환가격
0,경상남도 진주시 가좌동,1***,12,12.0,202003,5,1500,25,2007,진주대로404번길,128.105889,35.157466,2020,3,경상남도 진주시 가좌동 진주대로404번길,4.8,7750
1,경상남도 진주시 가좌동,1***,8,15.0,202003,11,500,40,2016,가좌길78번길,128.105857,35.157800,2020,3,경상남도 진주시 가좌동 가좌길78번길,4.8,10500
5,경상남도 진주시 가좌동,5**,12,19.8,202003,14,200,36,2011,내동로348번길,128.106121,35.158214,2020,3,경상남도 진주시 가좌동 내동로348번길,4.8,9200
6,경상남도 진주시 가좌동,1***,8,20.0,202003,26,300,30,2016,가좌길48번길,128.105857,35.157800,2020,3,경상남도 진주시 가좌동 가좌길48번길,4.8,7800
7,경상남도 진주시 가좌동,1***,12,20.1,202003,8,300,30,2014,진주대로404번길,128.105966,35.156961,2020,3,경상남도 진주시 가좌동 진주대로404번길,4.8,7800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5248,경상남도 진주시 호탄동,6**,25,30.0,202011,26,1000,30,2017,호탄길10번길,128.106121,35.158214,2020,11,경상남도 진주시 호탄동 호탄길10번길,4.7,8660
5249,경상남도 진주시 호탄동,6**,8,33.0,202011,14,200,33,2009,호탄길13번길,128.105966,35.156961,2020,11,경상남도 진주시 호탄동 호탄길13번길,4.7,8626
5250,경상남도 진주시 호탄동,6**,8,33.0,202011,16,1000,40,2016,호탄길34번길,128.106121,35.158214,2020,11,경상남도 진주시 호탄동 호탄길34번길,4.7,11213
5251,경상남도 진주시 호탄동,6**,12,33.0,202011,19,100,23,2002,호탄길21번길,128.106121,35.158214,2020,11,경상남도 진주시 호탄동 호탄길21번길,4.7,5972


## 지도

In [40]:
import math as m

def distance(x1, x2, y1, y2):
    radius = 6371 # 지구 반지름(km)
    toRadian = m.pi / 180

    deltaLatitude = abs(x1 - x2) * toRadian
    deltaLongitude = abs(y1 - y2) * toRadian

    sinDeltaLat = m.sin(deltaLatitude / 2)
    sinDeltaLng = m.sin(deltaLongitude / 2)
    squareRoot = m.sqrt(sinDeltaLat * sinDeltaLat + m.cos(x1 * toRadian) * m.cos(x2 * toRadian) * sinDeltaLng * sinDeltaLng)

    distance = 2 * radius * m.asin(squareRoot)

    return distance

In [41]:
gnu_lat = 35.1530444
gnu_long = 128.1010899

gnu_dist = []
for long, lat in zip(gajwa["경도"], gajwa["위도"]):
    d = distance(gnu_lat, lat, gnu_long, long)
    gnu_dist.append(d)
gajwa["gnu_distance"] = gnu_dist

In [42]:
def remove_out(dataframe, remove_col):
    dff = dataframe
    for k in remove_col:
        level_1q = dff[k].quantile(0.25)
        level_3q = dff[k].quantile(0.75)
        IQR = level_3q - level_1q
        rev_range = 3  # 제거 범위 조절 변수
        dff = dff[(dff[k] <= level_3q + (rev_range * IQR)) & (dff[k] >= level_1q - (rev_range * IQR))]
        dff = dff.reset_index(drop=True)
    return dff

gajwa = remove_out(gajwa, ["전세전환가격", "gnu_distance"])

gajwa

,시군구,번지,도로조건,계약면적(㎡),계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명,경도,위도,year,month,location,전월세전환율,전세전환가격,gnu_distance
0,경상남도 진주시 가좌동,1***,12,12.0,202003,5,1500,25,2007,진주대로404번길,128.105889,35.157466,2020,3,경상남도 진주시 가좌동 진주대로404번길,4.8,7750,0.657356
1,경상남도 진주시 가좌동,1***,8,15.0,202003,11,500,40,2016,가좌길78번길,128.105857,35.157800,2020,3,경상남도 진주시 가좌동 가좌길78번길,4.8,10500,0.683656
2,경상남도 진주시 가좌동,5**,12,19.8,202003,14,200,36,2011,내동로348번길,128.106121,35.158214,2020,3,경상남도 진주시 가좌동 내동로348번길,4.8,9200,0.734594
3,경상남도 진주시 가좌동,1***,8,20.0,202003,26,300,30,2016,가좌길48번길,128.105857,35.157800,2020,3,경상남도 진주시 가좌동 가좌길48번길,4.8,7800,0.683656
4,경상남도 진주시 가좌동,1***,12,20.1,202003,8,300,30,2014,진주대로404번길,128.105966,35.156961,2020,3,경상남도 진주시 가좌동 진주대로404번길,4.8,7800,0.621416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065,경상남도 진주시 호탄동,6**,25,30.0,202011,26,1000,30,2017,호탄길10번길,128.106121,35.158214,2020,11,경상남도 진주시 호탄동 호탄길10번길,4.7,8660,0.734594
3066,경상남도 진주시 호탄동,6**,8,33.0,202011,14,200,33,2009,호탄길13번길,128.105966,35.156961,2020,11,경상남도 진주시 호탄동 호탄길13번길,4.7,8626,0.621416
3067,경상남도 진주시 호탄동,6**,8,33.0,202011,16,1000,40,2016,호탄길34번길,128.106121,35.158214,2020,11,경상남도 진주시 호탄동 호탄길34번길,4.7,11213,0.734594
3068,경상남도 진주시 호탄동,6**,12,33.0,202011,19,100,23,2002,호탄길21번길,128.106121,35.158214,2020,11,경상남도 진주시 호탄동 호탄길21번길,4.7,5972,0.734594


In [43]:
import folium

m = folium.Map(location=[gnu_lat, gnu_long], zoom_start=16)

for lo, la, in zip(gajwa.경도, gajwa.위도):
    folium.Marker([la, lo]).add_to(m)

m

In [44]:
# m = folium.Map(location=[gajwa.위도.max(),gajwa.경도.min()], zoom_start=16)
# folium.Marker([gajwa.위도.max(),gajwa.경도.min()]).add_to(m)
# m

In [45]:
gajwa_data = pd.DataFrame.drop(gajwa, columns=["시군구","계약년월", "번지", "도로명", "계약일", "보증금(만원)", "월세(만원)", "location", "경도", "위도", "전월세전환율", "gnu_distance"], axis=1)

#gajwa_data.distance = gajwa_data.distance.transform(lambda x: (x - x.mean()) / x.std())
# gajwa_data.전세전환가격 = gajwa_data.전세전환가격.transform(lambda x: (x - x.mean()) / x.std())
gajwa_data

,도로조건,계약면적(㎡),건축년도,year,month,전세전환가격
0,12,12.0,2007,2020,3,7750
1,8,15.0,2016,2020,3,10500
2,12,19.8,2011,2020,3,9200
3,8,20.0,2016,2020,3,7800
4,12,20.1,2014,2020,3,7800
...,...,...,...,...,...,...
3065,25,30.0,2017,2020,11,8660
3066,8,33.0,2009,2020,11,8626
3067,8,33.0,2016,2020,11,11213
3068,12,33.0,2002,2020,11,5972


# 머신러닝

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
import lightgbm as lgb
import catboost
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [47]:
X = gajwa_data.drop(["전세전환가격"], axis=1)
y = gajwa_data['전세전환가격']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1645)

scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.fit_transform(X_test)

minmaxScaler = MinMaxScaler()
X_train_standardized2 = minmaxScaler.fit_transform(X_train)
X_test_standardized2 = minmaxScaler.fit_transform(X_test)

ValueError: could not convert string to float: '진주대로404번길'

## 선형회귀분석

In [ ]:
# lr = LinearRegression()
# lr.fit(X_train, y_train)
# y_preds = lr.predict(X_test)

# mse = mean_squared_error(y_test, y_preds)
# rmse = np.sqrt(mse)

# print(f"MSE : {mse:.3f}", f"RMSE: {rmse:.3f}")
# print(f"variance score : {r2_score(y_test, y_preds):.3f}")

In [ ]:
# lr.fit(X_train_standardized, y_train)
# lr.score(X_test_standardized, y_test)

## Ridge

In [ ]:
# R_model = Ridge()
# R_model.fit(X_train, y_train)
# print(R_model.score(X_test, y_test))

# R_model.fit(X_train_standardized, y_train)
# R_model.score(X_test_standardized, y_test)

## Lasso

In [ ]:
# L_model = Lasso(alpha=0.5)
# L_model.fit(X_train, y_train)
# print(L_model.score(X_test, y_test))

# L_model.fit(X_train_standardized, y_train)
# L_model.score(X_test_standardized, y_test)

## ElasticNet

In [ ]:
# ela = ElasticNet(alpha=0.5)
# ela.fit(X_train, y_train)
# print(ela.score(X_test, y_test))

# ela.fit(X_train_standardized, y_train)
# ela.score(X_test_standardized, y_test)

## 랜덤포레스트

In [ ]:
randomforest = RandomForestRegressor(random_state=0, n_jobs=-1)
randomforest_model = randomforest.fit(X_train, y_train)
print(randomforest_model.score(X_test, y_test))

randomforest_model = randomforest.fit(X_train_standardized, y_train)
print(randomforest_model.score(X_test_standardized, y_test))

0.43498909591531887
0.31542870581722815


In [ ]:
gbm = GradientBoostingRegressor()

gbm.fit(X_train, y_train)
print(gbm.score(X_test, y_test))
gbm.fit(X_train_standardized, y_train)
print(gbm.score(X_test_standardized, y_test))

0.4092474710095567
0.28521489993571547


## LightGBM

In [ ]:
lightgbm = lgb.LGBMRegressor()

lightgbm.fit(X_train, y_train)
print(lightgbm.score(X_test, y_test))

lightgbm.fit(X_train_standardized, y_train)
print(lightgbm.score(X_test_standardized, y_test))

0.43841340982610033
0.31503167545407873


## XGBoost

In [ ]:
xgboost = XGBRegressor(n_estimators=50)
xgboost.fit(X_train, y_train)
print(xgboost.score(X_test, y_test))

xgboost.fit(X_train_standardized, y_train)
xgboost.score(X_test_standardized, y_test)

0.41476352666671157


/Users/chance/opt/anaconda3/envs/py3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.24924051983107265

## CatBoost

In [ ]:
cb = catboost.CatBoostRegressor()
cb.fit(X_train_standardized, y_train)
cb.score(X_test_standardized, y_test)

Learning rate set to 0.043656
0:	learn: 2228.4671014	total: 1.46ms	remaining: 1.46s
1:	learn: 2193.6870823	total: 1.94ms	remaining: 970ms
2:	learn: 2163.2623964	total: 2.36ms	remaining: 785ms
3:	learn: 2132.2532310	total: 3.14ms	remaining: 782ms
4:	learn: 2105.1357268	total: 3.4ms	remaining: 677ms
5:	learn: 2076.7689187	total: 3.91ms	remaining: 647ms
6:	learn: 2050.3566912	total: 4.3ms	remaining: 611ms
7:	learn: 2025.5324165	total: 4.73ms	remaining: 587ms
8:	learn: 2000.9377145	total: 5.13ms	remaining: 564ms
9:	learn: 1979.1133798	total: 5.78ms	remaining: 572ms
10:	learn: 1957.8898515	total: 6.17ms	remaining: 555ms
11:	learn: 1938.0632715	total: 6.62ms	remaining: 545ms
12:	learn: 1920.2182316	total: 7.21ms	remaining: 547ms
13:	learn: 1901.8524847	total: 7.68ms	remaining: 541ms
14:	learn: 1885.9387158	total: 8.15ms	remaining: 535ms
15:	learn: 1871.8124408	total: 8.49ms	remaining: 522ms
16:	learn: 1857.3015867	total: 8.96ms	remaining: 518ms
17:	learn: 1844.4867072	total: 9.38ms	remaining

0.30414107156169345

In [ ]:
gajwa_data.corr()

,도로조건,계약면적(㎡),건축년도,year,month,전세전환가격
도로조건,1.000000,0.113570,-0.136373,-0.129810,-0.012779,0.025288
계약면적(㎡),0.113570,1.000000,-0.084219,-0.234432,0.077863,0.254928
건축년도,-0.136373,-0.084219,1.000000,0.113643,-0.042512,0.065055
year,-0.129810,-0.234432,0.113643,1.000000,-0.009450,0.396707
month,-0.012779,0.077863,-0.042512,-0.009450,1.000000,0.057014
전세전환가격,0.025288,0.254928,0.065055,0.396707,0.057014,1.000000
